In [1]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim 


In [2]:
df_org=pd.read_csv("Ilce_Org_Dagilimi.csv")
df_bolge=pd.read_csv("Şehir-Bolge.csv")
df_oy=pd.read_csv("Ilce_Oy_Oranlari.csv")
df_egitim=pd.read_csv("SecimSonuclari_EgitimDurumu.csv")
df_egitim_map=pd.read_csv("EğitimDurumuMapping.csv")
df_org.head()

,İlçe,Adana,Adıyaman,Afyonkarahisar,Ağrı,Aksaray,Amasya,Ankara,Antalya,Ardahan,...,Sivas,Tekirdağ,Tokat,Trabzon,Tunceli,Uşak,Van,Yalova,Yozgat,Zonguldak
0,Adalar,106,45,44,59,15,59,145,42,63,...,412,80,402,272,54,21,1036,156,70,42
1,Arnavutköy,1205,4360,569,7296,1315,4163,721,209,8439,...,7890,947,8947,7977,376,151,7566,127,2426,2471
2,Ataşehir,4212,1577,1633,3502,1115,4508,5218,1401,5409,...,41836,1693,10389,10691,4044,685,1639,643,4713,4569
3,Avcılar,2773,4054,1273,6758,2384,6924,2316,870,13154,...,15698,4492,23551,9127,4116,423,7623,553,11043,2876
4,Bağcılar,2804,25561,1067,5421,1822,13949,1689,844,11109,...,37906,3434,25800,13805,3357,1162,15663,283,8161,6373


In [3]:
df_bolge.head()

,Bölge,Şehir
0,Akdeniz,Adana
1,Akdeniz,Antalya
2,Akdeniz,Burdur
3,Akdeniz,Hatay
4,Akdeniz,Isparta


In [4]:
df_org=pd.melt(df_org, id_vars=['İlçe'], value_vars=df_org.columns[1:]).rename(columns={"variable":"Şehir","value":"Nüfus"})

In [5]:
df_tmp=pd.merge(df_org,df_bolge,on="Şehir")

In [6]:
df_tmp.head()

,İlçe,Şehir,Nüfus,Bölge
0,Adalar,Adana,106,Akdeniz
1,Arnavutköy,Adana,1205,Akdeniz
2,Ataşehir,Adana,4212,Akdeniz
3,Avcılar,Adana,2773,Akdeniz
4,Bağcılar,Adana,2804,Akdeniz


In [7]:
df_egitim.head()

,İlçe,Okuma yazma bilmeyen,Okuma yazma bilen fakat bir okul bitirmeyen,İlkokul,İlköğretim,Ortaokul veya dengi okul,Lise veya dengi okul,Yüksekokul veya fakülte,Yüksek lisans,Doktora,Bilinmeyen,Toplam
0,Adalar,229,350,2248,703,1193,3422,2872,465,149,336,11967
1,Arnavutköy,6771,10346,4822,41551,15084,30219,14559,933,124,1179,168986
2,Ataşehir,7471,791,54952,34388,26185,80064,77643,12878,2468,2487,306446
3,Avcılar,6541,10002,60842,46084,29595,85543,55042,4754,666,3217,302286
4,Bahçelievler,10614,13935,88413,61524,4121,115886,82914,8232,1231,3398,427357


In [8]:
df_egitim_map

,Sınıf,Education Status 1,Education Status 2
0,Okuma yazma bilmeyen,Un-Trained,Un-Trained
1,Okuma yazma bilen fakat bir okul bitirmeyen,Un-Trained,Un-Trained
2,İlkokul,Un-Trained,Un-Trained
3,İlköğretim,Un-Trained,Un-Trained
4,Ortaokul veya dengi okul,High-School,Un-Trained
5,Lise veya dengi okul,High-School,Trained
6,Yüksekokul veya fakülte,Undergraduate Education,Trained
7,Yüksek lisans,Graduate,Trained
8,Doktora,Graduate,Trained
9,Bilinmeyen,Un-Trained,Un-Trained


In [9]:
df_egitim=pd.read_csv("SecimSonuclari_EgitimDurumu.csv")
df_egitim_map=pd.read_csv("EğitimDurumuMapping.csv")
df_longlat=pd.read_csv("Ilce_Lat_long.csv")
df_egitim=pd.melt(df_egitim,id_vars=["İlçe"],value_vars=df_egitim.columns[1:]).rename(columns={"variable":"Sınıf","value":"Kişi"})
df_egitim=pd.merge(df_egitim,df_egitim_map,on="Sınıf")
df_egitim=df_egitim.groupby(["İlçe","Education Status 2"], as_index=False)["Kişi"].sum()
df_egitim=df_egitim.pivot(index="İlçe",columns="Education Status 2",values="Kişi").reset_index()
df_egitim.rename_axis(None, axis = 1,inplace=True)
#Eğitimli ve eğitimsiz kitlelerin ilçelere göre oranını bulacağım.
df_egitim["Trained%"]=df_egitim["Trained"]/(df_egitim["Trained"]+df_egitim["Un-Trained"])
df_egitim["Un-Trained%"]=df_egitim["Un-Trained"]/(df_egitim["Trained"]+df_egitim["Un-Trained"])
df_egitim=pd.merge(df_egitim,df_longlat,on="İlçe")

In [10]:
df_egitim.head()

,İlçe,Trained,Un-Trained,Trained%,Un-Trained%,Latitude,Longitude
0,Adalar,6908,5059,0.577254,0.422746,40.86862,29.12765
1,Arnavutköy,45835,79753,0.364963,0.635037,41.18032,28.74310
2,Ataşehir,173053,126274,0.578140,0.421860,40.98968,29.09217
3,Avcılar,146005,156281,0.483003,0.516997,40.97844,28.72365
4,Bahçelievler,208263,182005,0.533641,0.466359,41.01146,28.84362


## İlçelerin koordinatlarını bulalım

In [11]:
def get_coordinate(address): # Returns the coordinates of the given district
    ilce=replace_eng_chr(address)
    address="Turkey,Istanbul,"+ilce+","+ilce+",Istanbul"
    geolocator = Nominatim(user_agent=address)
    geolocator.scheme()
    location = geolocator.geocode(address)
    latitude_res = location.latitude
    longitude_res = location.longitude
    return latitude_res,longitude_res

In [12]:
def replace_eng_chr(text): # Replace Turkish Char To English
    text=text.replace("ğ","g")
    text=text.replace("Ğ","G")
    text=text.replace("ı","i")
    text=text.replace("İ","I")
    text=text.replace("ö","o")
    text=text.replace("Ö","O")
    text=text.replace("ü","u")
    text=text.replace("Ü","U")
    text=text.replace("ş","s")
    text=text.replace("Ş","S")
    text=text.replace("ç","c")
    text=text.replace("Ç","C")
    return text

In [13]:
#df_egitim["Latitude"]=df_egitim["İlçe"].map(get_coordinate)
#df_egitim["Latitude"], df_egitim["Longitude"] = zip(*df_egitim["İlçe"].map(get_coordinate))

In [14]:
df_egitim.head()
#df_egitim.to_csv("Ilce_Lat_long.csv")


,İlçe,Trained,Un-Trained,Trained%,Un-Trained%,Latitude,Longitude
0,Adalar,6908,5059,0.577254,0.422746,40.86862,29.12765
1,Arnavutköy,45835,79753,0.364963,0.635037,41.18032,28.74310
2,Ataşehir,173053,126274,0.578140,0.421860,40.98968,29.09217
3,Avcılar,146005,156281,0.483003,0.516997,40.97844,28.72365
4,Bahçelievler,208263,182005,0.533641,0.466359,41.01146,28.84362


In [27]:
get_coordinate("12")

TypeError: 'str' object is not callable

In [22]:
map_istanbul = folium.Map(location=[get_coordinate("İstanbul")[0],get_coordinate("İstanbul")[1]],zoom_start=10)

TypeError: 'str' object is not callable

In [21]:
map_istanbul = folium.Map(location=[get_coordinate("İstanbul")[0],get_coordinate("İstanbul")[1]],zoom_start=10)

for lat,lng,ilce in zip(df_egitim['Latitude'],df_egitim['Longitude'],df_egitim['İlçe']):
    label = replace_eng_chr(ilce)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [float(lat),float(lng)],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_istanbul)

map_istanbul

TypeError: 'str' object is not callable

In [17]:
get_coordinate("Turkey,Istanbul,Arnavutköy,Arnavutköy")

TypeError: 'str' object is not callable

In [ ]:
get_coordinate("Turkey,Istanbul,Arnavutköy,Arnavutköy")

In [19]:
ilce="Arnavutköy"
address=ilce+" Merkez,"+ilce+",Istanbul"
print(address)
geolocator = Nominatim(user_agent="Test")
location = geolocator.geocode(address)
latitude_res = location.latitude
longitude_res = location.longitude
print(latitude_res , longitude_res )

Arnavutköy Merkez,Arnavutköy,Istanbul
41.181092 28.7367643


In [18]:
Ilce="Turkey,Istanbul,Arnavutköy"
txt_koor=Ilce+ ","+Ilce
Koor=get_coordinate("Turkey,Istanbul,Arnavutköy,Arnavutköy")
map_bah = folium.Map(location=[Koor[0],Koor[1]],zoom_start=10)
label = Ilce
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
[Koor[0],Koor[1]],
radius=5,
popup=label,
color='blue',
fill=True,
fill_color='#3186cc',
fill_opacity=0.7,
parse_html=False).add_to(map_bah)
map_bah

TypeError: 'str' object is not callable

In [20]:
import requests
import json
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];[admin_level=6];
(node["amenity"="arnavutköy"](area);
 way["amenity"="arnavutköy"](area);
 rel["amenity"="arnavutköy"](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data